# Loading model

**‼️  Import mediapipe before tesorflow due to runtime issue**


In [1]:
import mediapipe as mp
import tensorflow as tf

In [2]:
import os

In [3]:
model_filename = "keypoint_classifier.hdf5"
model_save_path = os.path.join('model', model_filename)

In [4]:
DIRECTION = {
    'up': 0,
    'down': 1,
    'back': 2,
    'forward': 3,
    'left': 4,
    'right': 5
}
INVERSE_DIRECTION = {val: key for key, val in DIRECTION.items()}

In [5]:
model = tf.keras.models.load_model(model_save_path)

2021-12-09 15:13:21.835218: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [6]:
model.weights

[<tf.Variable 'dense/kernel:0' shape=(42, 32) dtype=float32, numpy=
 array([[ 0.28536266, -0.17170145,  0.01691386, ...,  0.10488549,
          0.08943686, -0.16523671],
        [ 0.58007896,  0.18528442, -0.20502406, ..., -0.4396787 ,
         -0.14293262, -0.28677806],
        [-0.00782894,  0.0783958 , -0.12761739, ...,  0.05299959,
         -0.277816  ,  0.3873113 ],
        ...,
        [-0.05804154, -0.06494728,  0.0198838 , ...,  0.20319828,
          0.22082582,  0.00704322],
        [ 0.1441247 , -0.20837139,  0.1039812 , ..., -0.3770678 ,
         -0.02897057, -0.00083756],
        [ 0.4402409 , -0.12186212, -0.0781737 , ...,  0.17160358,
         -0.0155257 ,  0.03420025]], dtype=float32)>,
 <tf.Variable 'dense/bias:0' shape=(32,) dtype=float32, numpy=
 array([ 0.21120265,  0.11878837,  0.04358736,  0.02852655,  0.        ,
         0.0698735 , -0.02003215, -0.01524948,  0.        ,  0.07415266,
         0.0214069 ,  0.        , -0.01923835, -0.01519306,  0.05001137,
       

# Showing live demo using cv2

In [7]:
import cv2
import numpy as np

In [8]:
def write_text_on_video(frame, direction, color=(255,255,255)):
    font = cv2.FONT_HERSHEY_SIMPLEX
    cv2.putText(frame,
               f"{direction}",
               (50, 50),
                font, 1, 
                color,
                2,
                cv2.LINE_4)

In [9]:
def extract_keypoints(landmark):
    if not landmark: return
    keypoints = []
    for data_point in enumerate(landmark[0].landmark):
        keypoints.extend(
            [data_point[1].x, data_point[1].y]
        )
    return keypoints
    

In [14]:
def add_hands_connection(frame, hands):
    image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    image.flags.writeable = False
    results = hands.process(image)
    image.flags.writeable = True
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
    
    if results.multi_hand_landmarks:
        for num, hand in enumerate(results.multi_hand_landmarks):
            mp_drawing.draw_landmarks(
                image, hand, 
                mp_hands.HAND_CONNECTIONS,
                mp_drawing.DrawingSpec(color=(121,22,78), thickness=2, circle_radius=4),
                mp_drawing.DrawingSpec(color=(121,44,250), thickness=2, circle_radius=2),
            )
    return image, results

In [11]:
def predict_label(keypoints):
    return INVERSE_DIRECTION[
        np.argmax(np.squeeze(
            model.predict([keypoints])
                  ))]

In [12]:
mp_drawing = mp.solutions.drawing_utils
mp_hands = mp.solutions.hands

In [15]:
cap = cv2.VideoCapture(0)
with mp_hands.Hands(min_detection_confidence=0.8, min_tracking_confidence=0.7) as hands:
    while cap.isOpened():
        ret, frame = cap.read()
        image, results = add_hands_connection(frame, hands)
        image = cv2.flip(image, 1)
        landmark_result = results.multi_hand_landmarks
        if landmark_result:
            keypoints = extract_keypoints(landmark_result)
            y_cap = predict_label(keypoints)
            write_text_on_video(image, y_cap)
        cv2.imshow("hand", image)
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
    cap.release()
    cv2.destroyAllWindows()
    cv2.waitKey(1)